In [1]:
# pip install transformers torch
# pip install sentencepiece
# pip install sacremoses

In [2]:
# Load model directly
from transformers import AutoTokenizer, MarianMTModel

src = "pl"  # source language
trg = "en"  # target language

model_name = f"Helsinki-NLP/opus-mt-{src}-{trg}"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Save the model and tokenizer locally
model.save_pretrained(f'./opus-mt-{src}-{trg}')
tokenizer.save_pretrained(f'./opus-mt-{src}-{trg}')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[63429]], 'forced_eos_token_id': 0}


('./opus-mt-pl-en/tokenizer_config.json',
 './opus-mt-pl-en/special_tokens_map.json',
 './opus-mt-pl-en/vocab.json',
 './opus-mt-pl-en/source.spm',
 './opus-mt-pl-en/target.spm',
 './opus-mt-pl-en/added_tokens.json')

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("translation", model=model_name)

trans_dct = pipe("Dzień dobry! Jak się masz?")[0]
print(trans_dct['translation_text'])

Good morning, how are you?


In [4]:
# Load the model and tokenizer
from transformers import AutoTokenizer, MarianMTModel

# Load the model and tokenizer from the local directory
tokenizer = AutoTokenizer.from_pretrained(f'./opus-mt-{src}-{trg}')
model = MarianMTModel.from_pretrained(f'./opus-mt-{src}-{trg}')
print(f"The model has {model.num_parameters():,d} parameters.")

# Function to translate text from Polish to English
def translate(text, model, tokenizer):
    # Tokenize the input text
    batch = tokenizer([text], return_tensors="pt")
    
    # Generate translation using the model
    generated_ids = model.generate(**batch)
    
    # Decode the translated tokens to get the translated text
    translated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return translated_text

The model has 77,138,944 parameters.


In [5]:
# Example usage
polish_text = "Dzień dobry! Jak się masz?"
english_translation = translate(polish_text, model, tokenizer)
print(english_translation) 

Good morning, how are you?


In [6]:
import re

polish_text = """
Litwo, Ojczyzno moja! ty jesteś jak zdrowie;
Ile cię trzeba cenić, ten tylko się dowie,
Kto cię stracił. Dziś piękność twą w całej ozdobie
Widzę i opisuję, bo tęsknię po tobie.

Panno święta, co Jasnej bronisz Częstochowy
I w Ostrej świecisz Bramie! Ty, co gród zamkowy
Nowogródzki ochraniasz z jego wiernym ludem!
Jak mnie dziecko do zdrowia powróciłaś cudem

Gdy od płaczącej matki, pod Twoją opiekę
Ofiarowany martwą podniosłem powiekę;
I zaraz mogłem pieszo, do Twych świątyń progu
Iść za wrócone życie podziękować Bogu

Tak nas powrócisz cudem na Ojczyzny łono!
Tymczasem, przenoś moją duszę utęsknioną
Do tych pagórków leśnych, do tych łąk zielonych,
Szeroko nad błękitnym Niemnem rozciągnionych;

Do tych pól malowanych zbożem rozmaitem,
Wyzłacanych pszenicą, posrebrzanych żytem;
Gdzie bursztynowy świerzop, gryka jak śnieg biała,
Gdzie panieńskim rumieńcem dzięcielina pała,
A wszystko przepasane jakby wstęgą, miedzą
Zieloną, na niej zrzadka ciche grusze siedzą.
"""
split_text = re.split('\n', polish_text)

for sentence in split_text:
    sentence = sentence.strip()
    if len(sentence) > 0:
        english_translation = translate(sentence, model, tokenizer)
        print(f'{sentence} -> {english_translation}\n')

Litwo, Ojczyzno moja! ty jesteś jak zdrowie; -> Lithuania, my homeland, you are like health; 

Ile cię trzeba cenić, ten tylko się dowie, -> As much as you need to be valued, this one will only find out, 

Kto cię stracił. Dziś piękność twą w całej ozdobie -> Who has lost you today your beauty in all its adornment 

Widzę i opisuję, bo tęsknię po tobie. -> I see and describe because I miss you. 

Panno święta, co Jasnej bronisz Częstochowy -> Miss Christmas, what are you defending Jasna Częstochowa? 

I w Ostrej świecisz Bramie! Ty, co gród zamkowy -> And in the Sighty One you glow the Gate! 

Nowogródzki ochraniasz z jego wiernym ludem! -> You protect Novgorod with his faithful people! 

Jak mnie dziecko do zdrowia powróciłaś cudem -> How did my baby recover by a miracle? 

Gdy od płaczącej matki, pod Twoją opiekę -> ♪ When from a crying mother ♪ ♪ Under your care ♪ 

Ofiarowany martwą podniosłem powiekę; -> Sacrificed dead I lifted up the eyelid; 

I zaraz mogłem pieszo, do Twych świ